# Zen Trader Simulator: Heiken-Ashi and Ichimoku-Driven Trade Insight

## 🌅 Introduction

This notebook is part of the Zen Trader project — an intuition-first, Ichimoku-inspired approach to understanding market flow.  
We simulate and reflect on trades using mood-based intent, Heiken Ashi reinforcement, and Ichimoku signals to guide our path with clarity and restraint.

We do not seek to beat the market, but to listen to it.

## 📦 Core Libraries and Environment

We begin by loading essential tools and activating our environment.

In [2]:
# Core libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Jupyter inline plotting
%matplotlib inline

# External tools (installed via requirements.txt)
import seaborn as sns
from tqdm import tqdm
from dotenv import load_dotenv
from ta.trend import IchimokuIndicator

# Load environment variables if needed
load_dotenv();

print('Completed')

Completed


## 📂 Dataset Loading

We load historical market data (EURUSD, 15-minute candles) for analysis and simulation.

In [3]:
def load_data_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    df = df.rename(columns={
        "Open": "open", "High": "high", "Low": "low", "Close": "close"
    })
    df = df[['open', 'high', 'low', 'close']].astype(float)
    return df

# Load and preview your dataset
df = load_data_from_csv("datasets/EURUSD_M15.csv")
df.head()


,open,high,low,close
0,1.12773,1.12826,1.12746,1.12809
1,1.12815,1.12851,1.12774,1.12788
2,1.12785,1.12822,1.12747,1.12808
3,1.12808,1.12834,1.12769,1.12769
4,1.12781,1.12827,1.12743,1.12776


## 🧼 Data Cleaning & Preparation

Before analysis and simulation, we prepare the raw dataset for reliability and consistency:

In [5]:
# Drop unused columns (optional, if applicable)
# df.drop(columns=['Unnamed: 0'], inplace=True)

# Drop rows with missing values (if any)
df.dropna(inplace=True)

# Reset index after cleaning (optional)
df.reset_index(drop=True, inplace=True)

## 🌥️ Generate Ichimoku Components

In [11]:
from ta.trend import IchimokuIndicator

# Initialize IchimokuIndicator
ichimoku = IchimokuIndicator(
    high=df['high'],
    low=df['low'],
    window1=9,
    window2=26,
    window3=52,
    visual=True,
    fillna=False
)

# Add Ichimoku layers
df['tenkan_sen'] = ichimoku.ichimoku_conversion_line()
df['kijun_sen'] = ichimoku.ichimoku_base_line()
df['senkou_span_a'] = ichimoku.ichimoku_a()
df['senkou_span_b'] = ichimoku.ichimoku_b()
df['chikou_span'] = df['close'].shift(-26)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,open,high,low,close,tenkan_sen,kijun_sen,senkou_span_a,senkou_span_b,chikou_span
0,1.13340,1.13371,1.13311,1.13334,1.132275,1.130100,1.235319,1.235317,1.13485
1,1.13477,1.13520,1.13471,1.13485,1.135380,1.135355,1.131188,1.130100,1.13540
2,1.13479,1.13563,1.13476,1.13521,1.135380,1.135595,1.132983,1.130950,1.13513
3,1.13507,1.13523,1.13472,1.13478,1.135380,1.135595,1.133235,1.131095,1.13554
4,1.13470,1.13495,1.13445,1.13450,1.135380,1.135200,1.133945,1.131630,1.13575


### 💾 Save Prepared Dataset

In [12]:
# Final integrity check
print("Final dataset shape:", df.shape)
print("Any NaN values?", df.isnull().values.any())

# Save if clean
if not df.isnull().values.any():
    df.to_csv("datasets/EURUSD_M15_ichimoku.csv", index=False)
    print("✅ Saved cleaned and enriched dataset to: datasets/EURUSD_M15_ichimoku.csv")
else:
    print("⚠️ Dataset contains NaN values — not saved.")

Final dataset shape: (512475, 9)
Any NaN values? False
✅ Saved cleaned and enriched dataset to: datasets/EURUSD_M15_ichimoku.csv


## 📚 Zen Modules

Here we define the core modules used throughout the Zen system.  
Each cell includes one self-contained module: Heiken-Ashi, Ichimoku, mood analysis, intent generation, and more.

In [12]:
# Module: zen_ichimoku_weather.py
# A humble weather vane

import matplotlib
matplotlib.use("Agg")

#import oandapyV20.endpoints.instruments as instruments
import matplotlib.pyplot as plt

'''
def fetch_candles(client, instrument, granularity="M5", count=100):
    """
    Fetch historical candles from Oanda.
    """
    params = {
        "count": count,
        "granularity": granularity,
        "price": "M"  # Mid prices
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    client.request(r)
    candles = r.response.get('candles', [])
    
    parsed_candles = []
    for candle in candles:
        if candle['complete']:
            parsed_candles.append({
                "time": candle["time"],
                "open": float(candle["mid"]["o"]),
                "high": float(candle["mid"]["h"]),
                "low": float(candle["mid"]["l"]),
                "close": float(candle["mid"]["c"])
            })
    return parsed_candles
'''

def compute_ichimoku(candles):
    """
    Compute Ichimoku components from candles.
    """
    highs = [c['high'] for c in candles]
    lows = [c['low'] for c in candles]
    closes = [c['close'] for c in candles]

    tenkan_sen = []
    kijun_sen = []
    senkou_span_a = []
    senkou_span_b = []
    chikou_span = []

    for i in range(len(candles)):
        if i >= 9 - 1:
            high = max(highs[i-8:i+1])
            low = min(lows[i-8:i+1])
            tenkan_sen.append((high + low) / 2)
        else:
            tenkan_sen.append(None)

        if i >= 26 - 1:
            high = max(highs[i-25:i+1])
            low = min(lows[i-25:i+1])
            kijun_sen.append((high + low) / 2)
        else:
            kijun_sen.append(None)

        if i >= 26:
            chikou_span.append((i-26, closes[i]))
        else:
            chikou_span.append(None)

    for i in range(len(candles)):
        if i >= 26 - 1 and tenkan_sen[i] is not None and kijun_sen[i] is not None:
            span_a = (tenkan_sen[i] + kijun_sen[i]) / 2
            if i + 26 < len(candles):
                senkou_span_a.append((i + 26, span_a))

        if i >= 52 - 1:
            high = max(highs[i-51:i+1])
            low = min(lows[i-51:i+1])
            span_b = (high + low) / 2
            if i + 26 < len(candles):
                senkou_span_b.append((i + 26, span_b))

    return {
        "tenkan_sen": tenkan_sen,
        "kijun_sen": kijun_sen,
        "senkou_span_a": senkou_span_a,
        "senkou_span_b": senkou_span_b,
        "chikou_span": chikou_span
    }

def plot_ichimoku(candles, ichimoku_lines, title="Ichimoku Sketch"):
    """
    Plot Ichimoku components and price.
    """
    closes = [c['close'] for c in candles]
    x = list(range(len(closes)))

    plt.figure(figsize=(14,8))
    plt.plot(x, closes, label="Price", color="black", linewidth=1)

    plt.plot(x, ichimoku_lines['tenkan_sen'], label="Tenkan-sen (9)", color="blue", linestyle='--')
    plt.plot(x, ichimoku_lines['kijun_sen'], label="Kijun-sen (26)", color="red", linestyle='-.')

    if ichimoku_lines['senkou_span_a']:
        x_span_a = [i for (i,v) in ichimoku_lines['senkou_span_a']]
        y_span_a = [v for (i,v) in ichimoku_lines['senkou_span_a']]
        plt.plot(x_span_a, y_span_a, label="Senkou Span A", color="green")

    if ichimoku_lines['senkou_span_b']:
        x_span_b = [i for (i,v) in ichimoku_lines['senkou_span_b']]
        y_span_b = [v for (i,v) in ichimoku_lines['senkou_span_b']]
        plt.plot(x_span_b, y_span_b, label="Senkou Span B", color="orange")

    chikou_x = []
    chikou_y = []
    for v in ichimoku_lines['chikou_span']:
        if v is not None:
            chikou_x.append(v[0])
            chikou_y.append(v[1])
    if chikou_x:
        plt.plot(chikou_x, chikou_y, label="Chikou Span", color="purple")

    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Price")
    plt.legend()
    plt.grid(True)
    plt.gca().invert_xaxis()
    plt.show()


In [13]:
# Module: zen_heiken_assist.py

import pandas as pd

def compute_heiken_ashi(df):
    ha_df = pd.DataFrame(index=df.index)
    ha_df['HA_Close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
    ha_open = [(df['open'].iloc[0] + df['close'].iloc[0]) / 2]  # seed
    for i in range(1, len(df)):
        ha_open.append((ha_open[i-1] + ha_df['HA_Close'].iloc[i-1]) / 2)
    ha_df['HA_Open'] = ha_open
    ha_df['HA_High'] = df[['high', 'low']].join(ha_df[['HA_Open', 'HA_Close']]).max(axis=1)
    ha_df['HA_Low'] = df[['high', 'low']].join(ha_df[['HA_Open', 'HA_Close']]).min(axis=1)
    return ha_df

def ha_trend_strength(ha_df, lookback=3):
    """
    Returns +1 for bullish, -1 for bearish, 0 for neutral based on HA color consistency.
    """
    recent = ha_df.tail(lookback)
    bullish = (recent['HA_Close'] > recent['HA_Open']).sum()
    bearish = (recent['HA_Close'] < recent['HA_Open']).sum()
    if bullish == lookback:
        return 1
    elif bearish == lookback:
        return -1
    return 0


In [14]:
# Module: zen_intent_engine.py

from zen_mood_reader import market_mood
from zen_seasonal_wisdom import get_market_session

def get_intent(candles, cloud_top, cloud_bottom, now_utc=None):
    """
    Determines market intent based on Ichimoku placement, mood, and seasonal session context.

    candles: List of recent candle dicts (open, high, low, close)
    cloud_top: float - top of the Ichimoku cloud
    cloud_bottom: float - bottom of the Ichimoku cloud
    now_utc: optional datetime for session detection (default: current UTC time)

    Returns a dictionary with enriched intent and session metadata.
    """

    # === Market Session Awareness ===
    session_data = get_market_session(now_utc)
    session_name = session_data.get("session", "Unknown")
    session_mood = session_data.get("mood", "")
    session_notes = session_data.get("notes", "")

    # === Mood Analysis ===
    mood_report = market_mood(candles, cloud_top, cloud_bottom)

    mood = mood_report["mood"]
    recent_tk_cross = mood_report["recent_tk_cross"]
    cloud_breakout = mood_report["cloud_breakout"]
    entered_cloud = mood_report["entered_cloud"]
    prior_mood = mood_report["prior_mood"]

    latest_close = candles[-1]["close"]

    # === Basic Position-Based Intent ===
    if latest_close > cloud_top:
        raw_intent = "bullish_bias"
    elif latest_close < cloud_bottom:
        raw_intent = "bearish_bias"
    else:
        raw_intent = "neutral"

    # === Confidence Logic ===
    confidence = 0.5  # base confidence

    if mood in ["soaring", "plunging"]:
        confidence += 0.25
    elif mood in ["climbing from valley", "slipping from heights"]:
        confidence += 0.1
    elif mood == "foggy":
        confidence = 0.0
    elif mood == "wandering":
        confidence -= 0.05

    # Session adjustments
    if session_name == "Weekend":
        confidence = 0.0
    elif session_name == "Holiday":
        confidence *= 0.5
    elif session_name == "Friday Close":
        confidence *= 0.7

    # Signal boosts
    if recent_tk_cross:
        confidence += 0.05
    if cloud_breakout:
        confidence += 0.05

    confidence = min(1.0, max(0.0, confidence))  # clamp to [0, 1]
    should_trade = confidence >= 0.6

    return {
        "type": raw_intent,
        "confidence": confidence,
        "should_trade": should_trade,
        "mood": mood,
        "session": session_name,
        "session_mood": session_mood,
        "session_notes": session_notes,
        "mood_context": {
            "recent_tk_cross": recent_tk_cross,
            "cloud_breakout": cloud_breakout,
            "entered_cloud": entered_cloud,
            "prior_mood": prior_mood
        }
    }


In [15]:
# Module: zen_mood_reader.py

def market_mood(candles, cloud_top, cloud_bottom):
    """
    Assess the market mood and recent Ichimoku-related events.

    candles: List of dicts with keys: open, high, low, close
    cloud_top: float
    cloud_bottom: float

    Returns a dictionary with mood and recent contextual insights.
    """
    if len(candles) < 3:
        return {
            "mood": "insufficient_data",
            "recent_tk_cross": False,
            "cloud_breakout": None,
            "entered_cloud": None,
            "prior_mood": None
        }

    recent = candles[-5:]
    directions = []
    body_sizes = []
    upper_wicks = []
    lower_wicks = []
    mood_sequence = []

    for c in recent[-3:]:
        body = abs(c['close'] - c['open'])
        wick_up = c['high'] - max(c['close'], c['open'])
        wick_down = min(c['close'], c['open']) - c['low']

        body_sizes.append(body)
        upper_wicks.append(wick_up)
        lower_wicks.append(wick_down)

        if c['close'] > c['open']:
            directions.append("bull")
        elif c['close'] < c['open']:
            directions.append("bear")
        else:
            directions.append("neutral")

    latest_close = recent[-1]['close']
    prev_close = recent[-2]['close']
    cloud_position = None

    if latest_close > cloud_top:
        cloud_position = "above"
    elif latest_close < cloud_bottom:
        cloud_position = "below"
    else:
        cloud_position = "inside"

    avg_body = sum(body_sizes) / len(body_sizes)
    avg_wick = (sum(upper_wicks) + sum(lower_wicks)) / (2 * len(body_sizes))
    bull_count = directions.count("bull")
    bear_count = directions.count("bear")

    # Determine mood
    mood = "wandering"
    if cloud_position == "inside":
        mood = "foggy"
    elif bull_count >= 2:
        if cloud_position == "above":
            mood = "soaring" if avg_body > avg_wick else "drifting"
        elif cloud_position == "below":
            mood = "climbing from valley"
    elif bear_count >= 2:
        if cloud_position == "below":
            mood = "plunging" if avg_body > avg_wick else "sliding"
        elif cloud_position == "above":
            mood = "slipping from heights"

    # Track prior mood from earlier 3 candles (optional)
    prior_mood = None
    if len(candles) >= 6:
        prior_mood = market_mood(candles[:-3], cloud_top, cloud_bottom)["mood"]

    # Simple TK cross detection (placeholder logic)
    # This approximates a Tenkan/Kijun crossover by checking for directional candle flips
    recent_tk_cross = False
    for i in range(1, len(recent)):
        if (
            (recent[i-1]['close'] < recent[i-1]['open'] and recent[i]['close'] > recent[i]['open']) or
            (recent[i-1]['close'] > recent[i-1]['open'] and recent[i]['close'] < recent[i]['open'])
        ):
            recent_tk_cross = True
            break

    # Cloud breakout or re-entry check
    # Check if we moved completely out of or into the cloud between the last 2 closes
    cloud_breakout = None
    entered_cloud = None
    if len(recent) >= 2:
        prev = recent[-2]['close']
        if prev < cloud_bottom and latest_close > cloud_top:
            cloud_breakout = "above"
        elif prev > cloud_top and latest_close < cloud_bottom:
            cloud_breakout = "below"
        if (cloud_bottom <= prev <= cloud_top) and (latest_close < cloud_bottom or latest_close > cloud_top):
            entered_cloud = False
        elif (prev < cloud_bottom or prev > cloud_top) and (cloud_bottom <= latest_close <= cloud_top):
            entered_cloud = True

    return {
        "mood": mood,
        "recent_tk_cross": recent_tk_cross,
        "cloud_breakout": cloud_breakout,
        "entered_cloud": entered_cloud,
        "prior_mood": prior_mood
    }


## 🔍 Exploratory Data Visualization *(Coming Soon)*

Initial visual analysis of price action, Ichimoku structure, and Heiken Ashi overlay.

## 🧠 Intent + Signal Extraction *(Coming Soon)*

Determine trade intent and signal strength based on mood, cloud structure, and confidence logic.

## 🧪 Simulation Run

This section runs the full trade simulation engine using:
- Ichimoku-derived signals
- Mood and intent analysis
- Heiken Ashi trend confirmation
- Entry and exit logic including trailing stops and fallback exits

Simulation results are stored for later analysis.

## 🧩 Training Set Slices *(Coming Soon)*

We create structured slices of the dataset for machine learning experiments:
- Feature extraction: mood, cloud position, trend strength, entry conditions
- Labeling: outcome classification (win/loss), PnL ranges, exit reason
- Confidence calibration datasets (for ML or regression modeling)

These slices will help refine confidence scoring, pattern recognition, and potentially adaptive strategies.

## 📈 Results + Insights *(Coming Soon)*

Post-simulation analysis, including trade summaries, win/loss stats, confidence effectiveness, and insight extraction.

## 🧘 Reflections *(Optional)*

A space for journaling trade philosophies, emotional states, and learning moments along the way.